In [1]:
import os
import cv2
import torch
from nanodet.util import cfg, load_config, Logger

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = torch.device('cuda')

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [2]:
config_path = 'assets/nanodet-300/nanodet-300.pth_train_config.yml'
model_path = 'assets/nanodet-300/nanodet-300.pth'
image_path = 'dataset/night/20201201_000505.jpg'

load_config(cfg, config_path)
logger = Logger(-1, use_tensorboard=False)

In [3]:
from nanodet.model.arch import build_model
from nanodet.util import Logger, cfg, load_config, load_model_weight
from nanodet.data.transform import Pipeline
from nanodet.data.collate import naive_collate
from nanodet.data.batch_process import stack_batch_img
import numpy as np

class WrapperModel(torch.nn.Module):
    def __init__(self, cfg, model_path, logger, device="cuda:0"):
        super().__init__()

        self.cfg = cfg
        self.device = device
        
        self.num_classes = cfg['model']['arch']['head']['num_classes']
        self.reg_max = cfg['model']['arch']['head']['reg_max']
        model = build_model(cfg.model)
        ckpt = torch.load(model_path, map_location=lambda storage, loc: storage)
        load_model_weight(model, ckpt, logger)
        self.model = model.to(device).eval()

    def forward(self, tensor_img):
        if len(tensor_img.shape) == 4:

            preds = self.model(tensor_img)
            cls_scores = preds.split(
                [self.num_classes, 4 * (self.reg_max + 1)], dim=-1
            )[0]
            
            max_cls_scores = torch.max(cls_scores.sigmoid(), dim=1)[0]
            return max_cls_scores
    
wrapper = WrapperModel(cfg, model_path, logger, device=device)

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet Head.


In [4]:
img = cv2.imread(image_path)

raw_height = img.shape[0]
raw_width  = img.shape[1]
dst_width, dst_height = cfg.data.val.input_size
ResizeM = np.eye(3)
ResizeM[0, 0] *= dst_width / raw_width
ResizeM[1, 1] *= dst_height / raw_height

# scaling only
numpy_img_warped = cv2.warpPerspective(img, 
                                       ResizeM, 
                                       dsize=tuple(cfg.data.val.input_size), 
                                       flags = cv2.INTER_LINEAR, 
                                       borderMode = cv2.BORDER_CONSTANT)

# normalise
mean, std = cfg.data.val.pipeline["normalize"]
mean = np.array(mean, dtype=np.float32).reshape(1, 1, 3) / 255
std = np.array(std, dtype=np.float32).reshape(1, 1, 3) / 255
numpy_img_normalised = ((numpy_img_warped.astype(np.float32) / 255) - mean) / std

# numpy to pytorch
processed_tensor_img = torch.from_numpy(numpy_img_normalised.transpose(2, 0, 1)).to(device).type(torch.cuda.FloatTensor).unsqueeze(0)

# input_   = torch.cat([processed_tensor_img, processed_tensor_img])
input_ = processed_tensor_img
print(input_.shape)

torch.Size([1, 3, 320, 320])


In [5]:
print(wrapper(input_))

tensor([[0.2940]], device='cuda:0', grad_fn=<MaxBackward0>)


/home/reedless/.venv/captum-vas/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/reedless/.venv/captum-vas/lib/python3.8/site-packages/torch/nn/functional.py:3499: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


## Captum

In [6]:
from captum.attr import (Deconvolution, DeepLift, DeepLiftShap,
                         FeatureAblation, GradientShap, GuidedBackprop,
                         GuidedGradCam, InputXGradient, IntegratedGradients,
                         Occlusion, Saliency)

baseline = torch.zeros(input_.shape).to(device).type(torch.cuda.FloatTensor)

thres = 0.35
class_scores = wrapper(input_)
pred_class = 0

# ig = IntegratedGradients(wrapper)
# attributions, delta = ig.attribute(input_,
#                                     target=pred_class,
#                                     return_convergence_delta=True)
dl = DeepLift(wrapper)
attributions, delta = dl.attribute(input_, baseline, target=pred_class, return_convergence_delta=True)

/home/reedless/.venv/captum-vas/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/reedless/.venv/captum-vas/lib/python3.8/site-packages/torch/nn/functional.py:3499: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/home/reedless/.venv/captum-vas/lib/python3.8/site-packages/captum/_utils/gradient.py:56: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.

RuntimeError: A Module LeakyReLU(negative_slope=0.1, inplace=True) was detected that does not contain some of the input/output attributes that are required for DeepLift computations. This can occur, for example, if your module is being used more than once in the network.Please, ensure that module is being used only once in the network.

In [ ]:
attributions